In [85]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import json
import networkx
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, no_more_discontiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
import pandas
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from functools import partial
import gerrychain.tree_proposals as tp
from gerrychain.constraints.validity import within_percent_of_ideal_population

#NOTE: must have tqdm installed (pip install tqdm in terminal)

In [3]:
#NOTE: An island warning will pop up.
graph = Graph.from_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")
print("graph done")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/graph/graph.py:196: UserWarning: Found islands (degree-0 nodes). Indices of islands: {352, 394, 372, 351}
  "Found islands (degree-0 nodes). Indices of islands: {}".format(islands)


graph done


In [9]:
#data = json.dumps(networkx.readwrite.json_graph.adjacency_data(graph))
#It won't let me save graph due to a Polygon error. 

In [ ]:
#with open('/Users/caranix/Documents/alaska_graphAK_DATA.json') as f:
#        data = json.load(f)
#graph = networkx.readwrite.json_graph.adjacency_graph(data)

In [87]:
elections = [
    Election("PRES16", {"Democratic": "PRES16D", "Republican": "PRES16R"}),
    #Election("USH14", {"Democratic": "USH14D", "Republican": "USH14R"}),
    #Election("SEN16", {"Democratic": "SEN16D", "Republican": "SEN16R"}),
    #Election("GOV18", {"Democratic": "GOV18D", "Republican": "GOV18R"}),
    #Election("USH16", {"Democratic": "USH16D", "Republican": "USH16R"})
    #Election("USH18", {"Democratic": "USH18D", "Republican": "USH18R"})
]



In [17]:
my_updaters = {"population": updaters.Tally("POPULATION", alias="population")}

election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)


In [71]:
initial_partition = GeographicPartition(graph, assignment="HDIST", updaters=my_updaters)


In [88]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(tp.recom,
                   pop_col="POPULATION",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

In [74]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

In [100]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        # District populations must stay within 3% of equality 
        constraints.within_percent_of_ideal_population(initial_partition, .03),
        compactness_bound, no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=1000
)

#NOTE: The current Alaksa plan does not have population within 2%, I incearesed it to 3 percent population deviation.

#NOTE: Apparently, Alaska is not single flip contiguous? Islands might be the issue? 
#Changed to no_more_discontiguous, so it doesn't get any worse. 


In [105]:
#chain with status bar 
data = pandas.DataFrame(
    sorted(partition["PRES16"].percents("Democratic"))
    for partition in chain.with_progress_bar()
)

# This will take about 10 minutes.


KeyError: 394

In [106]:
#without status bar- gives same error as above

#data = pandas.DataFrame(
#    sorted(partition["PRES16"].percents("Democratic"))
#    for partition in chain
#)

In [108]:
# trying to set it up for multi election chain runs - I'll fix this later. 
chain_results = {"PRES16": []} #,
                   #"USH14": [],
                   #"SEN16": [],
                   #"SEN18": [],
                   #"USH16": []}, 
                   #"USH18": []}

for partition in chain:
    for key, _ in chain_results.items():
        chain_results[key].append(sorted(partition["AK"].percents(key))) 

KeyError: 'AK'

In [81]:
data = pandas.DataFrame(
     sorted(partition["PRES16"].percents("Democratic"))
     for partition in chain.with_progress_bar()
 )

#this was an old error that I was getting before the key error (incase it doesn't show up again, see below)

NetworkXPointlessConcept: ('Connectivity is undefined ', 'for the null graph.')

 18% 183/1000 [00:19<00:05, 145.04it/s]

---------------------------------------------------------------------------
NetworkXPointlessConcept                  Traceback (most recent call last)
<ipython-input-81-96676cfdcabd> in <module>
      1 data = pandas.DataFrame(
      2      sorted(partition["PRES16"].percents("Democratic"))
----> 3      for partition in chain.with_progress_bar()
      4  )

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py in __init__(self, data, index, columns, dtype, copy)
    380         elif isinstance(data, (list, types.GeneratorType)):
    381             if isinstance(data, types.GeneratorType):
--> 382                 data = list(data)
    383             if len(data) > 0:
    384                 if is_list_like(data[0]) and getattr(data[0], 'ndim', 1) == 1:

<ipython-input-81-96676cfdcabd> in <genexpr>(.0)
      1 data = pandas.DataFrame(
----> 2      sorted(partition["PRES16"].percents("Democratic"))
      3      for partition in chain.with_progress_bar()
      4  )

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py in __iter__(self, *args, **kwargs)
    219     def __iter__(self, *args, **kwargs):
    220         try:
--> 221             for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
    222                 # return super(tqdm...) will not catch exception
    223                 yield obj

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/_tqdm.py in __iter__(self)
   1020                 """), fp_write=getattr(self.fp, 'write', sys.stderr.write))
   1021 
-> 1022             for obj in iterable:
   1023                 yield obj
   1024                 # Update and possibly print the progressbar.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/chain.py in __next__(self)
     78             self.state.parent = None
     79 
---> 80             if self.is_valid(proposed_next_state):
     81                 proposed_next_state.accepted = self.accept(proposed_next_state)
     82                 if proposed_next_state.accepted:

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/constraints/validity.py in __call__(self, partition)
     37         # check each constraint function and fail when a constraint test fails
     38         for constraint in self.constraints:
---> 39             is_valid = constraint(partition)
     40             if is_valid is False:
     41                 return False

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/constraints/bounds.py in __call__(self, partition)
    135             return self.__call__(partition)
    136         else:
--> 137             return self.func(partition) >= self.bound
    138 
    139 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/constraints/contiguity.py in number_of_contiguous_parts(partition)
    178     """
    179     parts = partition.assignment.parts
--> 180     return sum(1 for part in parts if nx.is_connected(partition.subgraphs[part]))
    181 
    182 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gerrychain/constraints/contiguity.py in <genexpr>(.0)
    178     """
    179     parts = partition.assignment.parts
--> 180     return sum(1 for part in parts if nx.is_connected(partition.subgraphs[part]))
    181 
    182 

<decorator-gen-233> in is_connected(G)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/networkx/utils/decorators.py in _not_implemented_for(not_implement_for_func, *args, **kwargs)
     71             raise nx.NetworkXNotImplemented(msg)
     72         else:
---> 73             return not_implement_for_func(*args, **kwargs)
     74     return _not_implemented_for
     75 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/networkx/algorithms/components/connected.py in is_connected(G)
    159     if len(G) == 0:
    160         raise nx.NetworkXPointlessConcept('Connectivity is undefined ',
--> 161                                           'for the null graph.')
    162     return sum(1 for node in _plain_bfs(G, arbitrary_element(G))) == len(G)
    163 

NetworkXPointlessConcept: ('Connectivity is undefined ', 'for the null graph.')

